# Jumble Solver
The jumble puzzle is a common newspaper puzzle, it contains a series of anagrams that must be solved (see https://en.wikipedia.org/wiki/Jumble). To solve, one must solve each of the individual jumbles. The circled letters are then used to create an additional anagram to be solved. In especially difficult versions, some of the anagrams in the first set can possess multiple solutions. To get the final answer, it is important to know all possible anagrams of a given series of letters.



# Problem Statement
Your challenge is to solve the five Jumble puzzles using Spark, where it makes sense to do so. You must use Python. If the final puzzle has multiple possible answers, you are to include an algorithm to determine the most likely one. We have provided a dictionary where the "most common" English words are scored (1=most frequent, 9887=least frequent, 0=not scored due to infrequency of use). For each puzzle, produce the "most likely" (as you determine it) final anagram produced from solving all the other anagrams.

# Solution 
Before getting stat with code, we have to make sure about our programming environment in which I have executed this code and it's running really well. 
I have used following environment:

1.Ubuntu (18.04.2) installed in Virtualbox (6.0.8){64-bit}

2.Java 8

3.Used Pyspark in jupyternote book

# Approach
I have provided with Jumble pictures (Picture 1 to Picture 5) and "freq_dist.json". 
So, as the jumble words we can see in those 5 pictures provided with this assignment, I had to first built a .json out of those 5 pictures and named as "Puzzles.json".
Now, we put  "Puzzles.json" and "freq_dist.json" in a same folder, open jupyter notebook and startworking on code execution.  

# Result
I have attached result screenshots and .txt files(Puzzle numerber 1,Puzzle number 2,Puzzle number 3,Puzzle number 4,Puzzle 5) and  with the assignment.


# Import Libraries

In [1]:
import json
import re
import copy
from itertools import permutations, product
from operator import itemgetter
from pyspark import SparkConf, SparkContext
from pyspark.sql import Row, Column, SparkSession
import pyspark.sql.functions as SQF
import pyspark.sql.types as SQT



In [2]:
#conf = SparkConf().setMaster("local").setAppName("EXE_MAIN")
#sc = SparkContext(conf=conf)

spark = SparkSession.builder.appName("MySparkSQL").getOrCreate()

# Creating Functions

In [3]:
# ---------------UTIL FUNCTIONS--------------------------
def sortLettersUtil(word):
    return ''.join(sorted(word))


def getLettersAtIdxs(wrd, idxList):
    letters = []
    for n in idxList:
        letters.append(wrd[n])
    return letters


In [4]:
# ---------------DICTIONARY FUNCTIONS--------------------------
def createPyDictionary(dictionarySource):
  dictionary = json.load(open(dictionarySource, "rb"))
  for k, _ in dictionary.items():
    if dictionary[k] == 0:
      dictionary[k] = 9888
    return dictionary

def createDictionaryDf(dictionarySource):
    wordsDict = json.load(open(dictionarySource, "rb"))
    # Wordlist is a list of lists each containing the word and its frequency rating
    wordsList = []
    for word in wordsDict.keys():
      wordsList.append([word, wordsDict[word]])

    dictionarySchema = SQT.StructType([
        SQT.StructField('word', SQT.StringType(), False),
        SQT.StructField('freq', SQT.IntegerType(), False)
    ])
    df = spark.createDataFrame(wordsList, dictionarySchema)
    udf = SQF.UserDefinedFunction(sortLettersUtil, SQT.StringType())
    df = df.withColumn("alpha_sorted", udf("word"))
    df = df.replace(0, 9888, "freq")
    df = df.withColumn("wrd_len", SQF.length("word"))
    return df


def findAllMatches(alphaSortedWord, df):
    df = df.filter(df.alpha_sorted == alphaSortedWord)
    df = df.orderBy(df.freq.asc())
    df = df.select("word")
    matchesList = df.rdd.flatMap(lambda x: x).collect()
    return matchesList

# Puzzle Solving

In [5]:
# ---------------PUZZLE FUNCS /PUZZLE SOLVING WITH PYSPARK---------------------------
def loadPuzzles(puzzPath):
    puzzles = json.load(open(puzzPath, "rb"))
    return puzzles["puzzles"]


def createPuzzles(puzzSource):
    puzzlesSchema = SQT.StructType([
        SQT.StructField('jumbles', SQT.ArrayType(
            SQT.MapType(SQT.StringType(), SQT.ArrayType(SQT.ShortType(), False), False), False)),
        SQT.StructField('id', SQT.ShortType(), False),
        SQT.StructField('finalWordLengths',
                        SQT.ArrayType(SQT.ShortType()), False)
    ])
    df = spark.createDataFrame(puzzSource, puzzlesSchema)
    return df


def getSinglePuzzleData(allPuzzlesDf, puzz_id):
    singlePuzzle = allPuzzlesDf.filter(allPuzzlesDf.id == puzz_id).rdd.flatMap(lambda x: x).collect()
    return singlePuzzle


def getAllPuzzleData(allPuzzlesDf):
    ids = allPuzzlesDf.select('id')
    ids = ids.rdd.flatMap(lambda x: x).collect()
    puzzDataList = []
    for i in ids:
        puzzDataList.append(getSinglePuzzleData(allPuzzlesDf, i))
    return puzzDataList

def createSinglePuzzDf(puzzleData, dictionary):
    jumblesList = puzzleData[0]
    table = []
    for wrdmap in jumblesList:
        for key in wrdmap:
            srtd = sortLettersUtil(key)
            matches = findAllMatches(srtd, dictionary)
        circledLetters = []
        for wrd in matches:
            circledLetters.append(getLettersAtIdxs(wrd, wrdmap[key]))
        tRow = (srtd, key, wrdmap[key], matches, circledLetters)
        table.append(tRow)
    rdd = sc.parallelize(table)
    puzzlesSchema = SQT.StructType([
        SQT.StructField('alpha_sorted', SQT.StringType(), False),
        SQT.StructField('scrambled', SQT.StringType(), False),
        SQT.StructField('idxs_list', SQT.ArrayType(
            SQT.ShortType(), False), False),
        SQT.StructField('matches_list', SQT.ArrayType(
            SQT.StringType(), False), False),
        SQT.StructField('letters_lists', SQT.ArrayType(
            SQT.ArrayType(SQT.StringType(), False), False), False)
    ])
    df = spark.createDataFrame(rdd, puzzlesSchema)
    return df



def createAllPuzzleDFs(batchDataList, dictionary):
    puzzlesDictionary = {}
    for p in batchDataList:
        puzzlesDictionary[p[1]] = {"df": createSinglePuzzDf(p, dictionary), "lengths": p[2]}
    return puzzlesDictionary



In [6]:
 #---------------PUZZLE SOLVING WITH PYTHON--------------------------
def loadPuzzles(puzzPath):
    puzzles = json.load(open(puzzPath, "rb"))
    return puzzles["puzzles"]    

def createPuzzles(puzzSource):
    puzzlesSchema = SQT.StructType([
        SQT.StructField('jumbles', SQT.ArrayType(
            SQT.MapType(SQT.StringType(), SQT.ArrayType(SQT.ShortType(), False), False), False)),
        SQT.StructField('id', SQT.ShortType(), False),
        SQT.StructField('finalWordLengths',
                        SQT.ArrayType(SQT.ShortType()), False)
    ])
    df = spark.createDataFrame(puzzSource, puzzlesSchema)
    return df


def getSinglePuzzleData(allPuzzlesDf, puzz_id):
    singlePuzzle = allPuzzlesDf.filter(allPuzzlesDf.id == puzz_id).rdd.flatMap(lambda x: x).collect()
    return singlePuzzle


def getAllPuzzleData(allPuzzlesDf):
    ids = allPuzzlesDf.select('id')
    ids = ids.rdd.flatMap(lambda x: x).collect()
    puzzDataList = []
    for i in ids:
        puzzDataList.append(getSinglePuzzleData(allPuzzlesDf, i))
    return puzzDataList

def createSinglePuzzDf(puzzleData, dictionary):
    jumblesList = puzzleData[0]
    table = []
    for wrdmap in jumblesList:
        for key in wrdmap:
            srtd = sortLettersUtil(key)
            matches = findAllMatches(srtd, dictionary)
        circledLetters = []
        for wrd in matches:
            circledLetters.append(getLettersAtIdxs(wrd, wrdmap[key]))
        tRow = (srtd, key, wrdmap[key], matches, circledLetters)
        table.append(tRow)
    rdd = sc.parallelize(table)
    puzzlesSchema = SQT.StructType([
        SQT.StructField('alpha_sorted', SQT.StringType(), False),
        SQT.StructField('scrambled', SQT.StringType(), False),
        SQT.StructField('idxs_list', SQT.ArrayType(
            SQT.ShortType(), False), False),
        SQT.StructField('matches_list', SQT.ArrayType(
            SQT.StringType(), False), False),
        SQT.StructField('letters_lists', SQT.ArrayType(
            SQT.ArrayType(SQT.StringType(), False), False), False)
    ])
    df = spark.createDataFrame(rdd, puzzlesSchema)
    return df



def createAllPuzzleDFs(batchDataList, dictionary):
    puzzlesDictionary = {}
    for p in batchDataList:
        puzzlesDictionary[p[1]] = {"df": createSinglePuzzDf(p, dictionary), "lengths": p[2]}
    return puzzlesDictionary


    
def PermutationsFromList(ll, length, pydictionary):
    perms = [''.join(i) for i in permutations(ll, r=length)]
    validPerms = set()
    for p in perms:
        if p in pydictionary:
            validPerms.add(p)
    sortedValidPerms = sorted(list(validPerms), key=lambda x: pydictionary[x])
    return sortedValidPerms


def createPermutations(ll, length, pydictionary, combosList=[]):
    results = []

    if (len(combosList) == 0):
      perms = PermutationsFromList(ll, length, pydictionary)
      for p in perms:
        newLetters = list(ll)
        for letter in p:
            newLetters.remove(letter)
        newPermItem = ([p], newLetters)
        results.append(newPermItem)
      return results

    else:
      for wrdList, lettersList in combosList:
        perms = perms = PermutationsFromList(lettersList, length, pydictionary)
        for p in perms:
          newWL = copy.copy(wrdList)
          newWL.append(p)
          newLetters = list(lettersList)
          for letter in p:
              newLetters.remove(letter)
          newPermItem = (newWL, newLetters)
          results.append(newPermItem)
      return results


def makePhrases(startingLettersList, wrdLensList, pydictionary):
  results = []
  index = 0
  for n in wrdLensList:
    print(n)
    if index == 0:
      result = createPermutations(startingLettersList, n, pydictionary)
    else:
      result = createPermutations(startingLettersList, n, pydictionary, results[-1])
    index += 1
    results.append(result)
  return results[-1]


def phrasesSortedByScore(phrasesList, pydictionary):
    results = []

    for tup in phrasesList:
        score = 0
        for wrd in tup[0]:
            wrdscore = pydictionary[wrd]
            score += wrdscore
        results.append((tup[0], score))
    sortedResults = sorted(results, key=itemgetter(1))
    return sortedResults[:10]


def printFile(filename, data):
    filenameext = filename+".txt"
    f = open(filenameext, "w+")
    f.write("The Top (10) solution(s) with score(s) for Puzzle")
    f.write(filename)
    f.write(" are as follows:\n")
    stringdata= str(data)
    f.write(stringdata)
    f.close()


def solvePuzzleBatchAndExport(puzzleDfDataObject, pydictionary):
    results = []
    for k, v in puzzleDfDataObject.items():
        df = v["df"]
        lengths = v["lengths"]
        df.show(10, False)

        mostLikeyLettersList = df.select('letters_lists').rdd.map(
            lambda lettersList: lettersList[0][0]).collect()
        ll = [item for sublist in mostLikeyLettersList for item in sublist]

        allPhrases = makePhrases(ll, lengths, pydictionary)
        sortedPhrases = phrasesSortedByScore(allPhrases, pydictionary)
        puzzlename = "Puzzle_" + str(k)
        results.append((puzzlename, sortedPhrases))

    for r in results:
        printFile(r[0], r[1])
    return results


# filepath strings
PUZZLE_SOURCE= "puzzles.json"
DICTIONARY_SOURCE= "freq_dict.json"

# creating dictionaries
PY_DICTIONARY = createPyDictionary(DICTIONARY_SOURCE)
DF_DICTIONARY = createDictionaryDf(DICTIONARY_SOURCE)

# creating puzzles
LOAD_PUZZLES = loadPuzzles(PUZZLE_SOURCE)
CREATE_PUZZLES = createPuzzles(LOAD_PUZZLES)
ALL_PUZZ_DATA_LIST = getAllPuzzleData(CREATE_PUZZLES)
CREATE_ALL_PUZZ_DFs = createAllPuzzleDFs(ALL_PUZZ_DATA_LIST, DF_DICTIONARY)

# solving puzzles
RESULTS = solvePuzzleBatchAndExport(CREATE_ALL_PUZZ_DFs, PY_DICTIONARY)

+------------+---------+---------+------------------------+---------------------------------+
|alpha_sorted|scrambled|idxs_list|matches_list            |letters_lists                    |
+------------+---------+---------+------------------------+---------------------------------+
|adgln       |nalgd    |[1, 3, 4]|[gland]                 |[[l, n, d]]                      |
|ajmor       |ramoj    |[2, 3]   |[major, joram, jarmo]   |[[j, o], [r, a], [r, m]]         |
|abcelm      |camble   |[0, 1, 3]|[becalm]                |[[b, e, a]]                      |
|aelrwy      |wraley   |[0, 2, 4]|[lawyer, yawler, warely]|[[l, w, e], [y, w, e], [w, r, l]]|
+------------+---------+---------+------------------------+---------------------------------+

3
4
4
+------------+---------+---------+------------------------+---------------------------------+
|alpha_sorted|scrambled|idxs_list|matches_list            |letters_lists                    |
+------------+---------+---------+-------------------

In [7]:
spark.stop()